In [6]:
from raid.utils import load_data

train_df = load_data(split="train",include_adversarial=False)

Human model will have NaN decoding duh!

In [7]:
import numpy as np
train_df['model'] = np.where(train_df['model'] == 'human',0,1)

In [8]:
train_df['model'].value_counts()

model
1    454614
0     13371
Name: count, dtype: int64

The class imbalance for Ai vs Human is 35:1. So to combat this we have to use class weighting. Adding this weight = (total number of samples)/(number of classes * number of samples)

In [8]:
train_df.head()

,id,adv_source_id,source_id,model,decoding,repetition_penalty,attack,domain,title,prompt,generation
0,e5e058ce-be2b-459d-af36-32532aaba5ff,e5e058ce-be2b-459d-af36-32532aaba5ff,e5e058ce-be2b-459d-af36-32532aaba5ff,0,NaN,NaN,none,abstracts,FUTURE-AI: Guiding Principles and Consensus Re...,NaN,The recent advancements in artificial intellig...
1,f95b107b-d176-4af5-90f7-4d0bb20caf93,f95b107b-d176-4af5-90f7-4d0bb20caf93,f95b107b-d176-4af5-90f7-4d0bb20caf93,0,NaN,NaN,none,abstracts,EdgeFlow: Achieving Practical Interactive Segm...,NaN,High-quality training data play a key role in ...
2,856d8972-9e3d-4544-babc-0fe16f21e04d,856d8972-9e3d-4544-babc-0fe16f21e04d,856d8972-9e3d-4544-babc-0fe16f21e04d,0,NaN,NaN,none,abstracts,Semi-supervised Contrastive Learning for Label...,NaN,The success of deep learning methods in medica...
3,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,0,NaN,NaN,none,abstracts,Combo Loss: Handling Input and Output Imbalanc...,NaN,Simultaneous segmentation of multiple organs f...
4,72c41b8d-0069-4886-b734-a4000ffca286,72c41b8d-0069-4886-b734-a4000ffca286,72c41b8d-0069-4886-b734-a4000ffca286,0,NaN,NaN,none,abstracts,Attention-Based 3D Seismic Fault Segmentation ...,NaN,Detection faults in seismic data is a crucial ...


In [9]:
# train_df = train_df.drop(
#     [
#         "id",
#         "decoding",
#         "domain",
#         "adv_source_id",
#         "source_id",
#         "repetition_penalty",
#         "attack",
#         "title",
#         "prompt"
#     ],
#     axis=1,
# )

train_df = train_df.rename(columns={
    "generation" : "content",
    "model" : "labels"
})

train_df.head()

,id,adv_source_id,source_id,labels,decoding,repetition_penalty,attack,domain,title,prompt,content
0,e5e058ce-be2b-459d-af36-32532aaba5ff,e5e058ce-be2b-459d-af36-32532aaba5ff,e5e058ce-be2b-459d-af36-32532aaba5ff,0,NaN,NaN,none,abstracts,FUTURE-AI: Guiding Principles and Consensus Re...,NaN,The recent advancements in artificial intellig...
1,f95b107b-d176-4af5-90f7-4d0bb20caf93,f95b107b-d176-4af5-90f7-4d0bb20caf93,f95b107b-d176-4af5-90f7-4d0bb20caf93,0,NaN,NaN,none,abstracts,EdgeFlow: Achieving Practical Interactive Segm...,NaN,High-quality training data play a key role in ...
2,856d8972-9e3d-4544-babc-0fe16f21e04d,856d8972-9e3d-4544-babc-0fe16f21e04d,856d8972-9e3d-4544-babc-0fe16f21e04d,0,NaN,NaN,none,abstracts,Semi-supervised Contrastive Learning for Label...,NaN,The success of deep learning methods in medica...
3,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,0,NaN,NaN,none,abstracts,Combo Loss: Handling Input and Output Imbalanc...,NaN,Simultaneous segmentation of multiple organs f...
4,72c41b8d-0069-4886-b734-a4000ffca286,72c41b8d-0069-4886-b734-a4000ffca286,72c41b8d-0069-4886-b734-a4000ffca286,0,NaN,NaN,none,abstracts,Attention-Based 3D Seismic Fault Segmentation ...,NaN,Detection faults in seismic data is a crucial ...


In [20]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

labels = train_df["labels"]

class_weights = compute_class_weight("balanced",classes=np.unique(labels),y=labels)

print(class_weights)

[17.5         0.51470588]


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import torch.nn as nn

weights = torch.tensor(class_weights, dtype=torch.float)

model_path = "distilbert/distilroberta-base"

loss_fn = nn.CrossEntropyLoss(weight=weights).to("cuda")

model = AutoModelForSequenceClassification.from_pretrained(model_path,num_labels=2).to("cuda")
tokeinzer = AutoTokenizer.from_pretrained(model_path)   

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilbert/distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

train_data, val_data = train_test_split(train_df, test_size=0.1, random_state=42, stratify=train_df["labels"])
train_dataset = Dataset.from_pandas(train_data)

val_data,test_data = train_test_split(val_data, test_size=0.5, random_state=42, stratify=val_data["labels"])
val_dataset = Dataset.from_pandas(val_data)
test_dataset = Dataset.from_pandas(test_data)

In [9]:
train_dataset

Dataset({
    features: ['id', 'adv_source_id', 'source_id', 'labels', 'decoding', 'repetition_penalty', 'attack', 'domain', 'title', 'prompt', 'content'],
    num_rows: 421186
})

In [11]:
train_dataset = train_dataset.remove_columns(["__index_level_0__"])
val_dataset = val_dataset.remove_columns(["__index_level_0__"])
test_dataset = test_dataset.remove_columns(["__index_level_0__"])

In [13]:
def preprocess_fn(batch):
    encodings = tokenizer(
        batch["content"], truncation=True, padding="max_length", max_length=512, return_tensors="pt"
    )
    return encodings


# train_dataset = train_dataset.map(preprocess_fn, batched=True)
# val_dataset = val_dataset.map(preprocess_fn, batched=True)
test_dataset = test_dataset.map(preprocess_fn, batched=True)

Map:   0%|          | 0/23400 [00:00<?, ? examples/s]

In [14]:
test_dataset

Dataset({
    features: ['id', 'adv_source_id', 'source_id', 'labels', 'decoding', 'repetition_penalty', 'attack', 'domain', 'title', 'prompt', 'content', 'input_ids', 'attention_mask'],
    num_rows: 23400
})

In [27]:
from transformers import TrainingArguments, Trainer


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False,**kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss
    
training_args = TrainingArguments(
    output_dir="./checkpoints",
    eval_strategy="epoch",
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=3e-5,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_dir="./logs",
    logging_steps=10,
    report_to="wandb",
    run_name="distilroberta-noadv-raid"
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokeinzer
)

/tmp/ipykernel_22631/3357938601.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(


In [28]:
trainer.train()

wandb: Currently logged in as: dattucodes (death-star) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.000200,0.122351


KeyError: "The `metric_for_best_model` training argument is set to 'eval_f1', which is not found in the evaluation metrics. The available evaluation metrics are: ['eval_loss']. Consider changing the `metric_for_best_model` via the TrainingArguments."

In [15]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_path = "./distilroberta-noadv-raid"
model = AutoModelForSequenceClassification.from_pretrained(model_path,num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilroberta-base")

In [32]:
from tqdm import tqdm


def inference(texts) -> list:
    predictions = []
    for text in tqdm(texts):
        inputs = tokenizer(text["content"], truncation=True, return_tensors="pt")
        outputs = model(**inputs)  
        probs = outputs.logits.softmax(dim=-1)
        _, fake = probs.detach().cpu().flatten().numpy().tolist()
        predictions.append({
            'id' : text['id'],
            'score' : fake
        })
    return predictions


preds = inference(test_dataset)

100%|██████████| 23400/23400 [11:26<00:00, 34.10it/s]


In [ ]:
from raid import run_evaluation

test_df = test_dataset.to_pandas()

result = run_evaluation(preds, test_df)

result

ValueError: Predictions are missing outputs for human-written texts in some domains.
In order to run evaluation, you must include predictions for human-written data in all domains.
To disable this, set per_domain_tuning=False in run_evaluation.